# Notes

code for producing GOIT pipelines summary stats, and for calculating landing page stats

this is saved as an Excel file, which Baird copies/pastes into the existing summary tables information on the drive here:
https://docs.google.com/spreadsheets/d/1OYH6D7c-D0FsL5GzBGijtkmvQCTkBUclj-UVoOieUFo/edit

In [1]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [2]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

## import data

In [3]:
#fuel_type = 'Gas'
fuel_type = 'Oil'
#fuel_type = 'NGL'

if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-Gas-'+str(datetime.date.today())+'.xlsx')
    fuel_options = ['Gas']
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-NGL-'+str(datetime.date.today())+'.xlsx')
    fuel_options = ['NGL', 
                    'NGL, oil products', 
                    'Oil, NGL', 
                    'Oil, NGL, naphtha']
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-Oil-'+str(datetime.date.today())+'.xlsx')
    fuel_options = ['Oil', 
                    'Oil, NGL', 
                    'Oil, NGL, naphtha']

In [4]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1WaBMIdfRWqSqXUw7_cKXo3RipyhPdnNN8flqEYfMZIA') # file to use for gas pipelines Dec 2023
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # CURRENT sheet

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
#pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')
country_ratios_df = country_ratios_df.loc[country_ratios_df.Wiki!='']

# remove empty cells for pipes, owners
if fuel_type=='Gas':
    pipes_df_orig = gas_pipes.copy()
if fuel_type in ['Oil','NGL']:
    pipes_df_orig = oil_pipes.copy()
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
#pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['Wiki']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Fuel.isin(fuel_options)]

owners_df_orig = owners_df_orig.loc[owners_df_orig['PipelineName']!='']
#owners_df_orig = owners_df_orig.loc[owners_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Status!='N/A']
owners_df_orig.set_index('ProjectID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [5]:
country_ratios_df = country_ratios_df.replace('--', numpy.nan)
owners_df_orig = owners_df_orig.replace('',numpy.nan)
owners_df_orig = owners_df_orig.replace('--',numpy.nan)
pipes_df_orig = pipes_df_orig.replace('--',numpy.nan)

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_88092/1118424270.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  country_ratios_df = country_ratios_df.replace('--', numpy.nan)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_88092/1118424270.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  owners_df_orig = owners_df_orig.replace('--',numpy.nan)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_88092/1118424270.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remo

In [6]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')

#region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
#region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

#region_df_touse = region_df_orig.copy()

In [7]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([('Africa',    'Northern Africa'),
            ('Africa', 'Sub-Saharan Africa')],
           names=['Region', 'SubRegion'])

## file names with regional specifics

### create country-specific dataframes for region, country_ratios_df, owners_df

In [8]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

### sum MergedKmByCountry and MergedKmByRegion

In [9]:
status_list = ['proposed', 
               'construction', 
               'shelved', 
               'cancelled', 
               'operating', 
               'idle', 
               'mothballed', 
               'retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [10]:
excel_status_list = ['proposed', 
                     'construction', 
                     'in development (proposed + construction)', 
                     'shelved', 
                     'cancelled', 
                     'operating', 
                     'idle', 
                     'mothballed', 
                     'retired']
excel_status_list_with_countries = ['Country']+excel_status_list

In [11]:
country_ratios_df_subset = country_ratios_df_touse.loc[country_ratios_df_touse['Fuel'].isin(fuel_options)]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['LengthMergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby(['Region','SubRegion'])['LengthMergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['in development (proposed + construction)'] = km_by_region[['proposed','construction']].sum(axis=1)
km_by_country['in development (proposed + construction)'] = km_by_country[['proposed','construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.names = ['Region','Subregion']
km_by_country.index.name = 'Country'

km_by_region.loc['Total',:] = km_by_region.sum(axis=0).values
km_by_country.loc['Total',:] = km_by_country.sum(axis=0).values

# drop all-zero rows
km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]

km_by_country.replace(0,'',inplace=True)
km_by_region.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
proposed
construction
shelved
cancelled
operating
idle
mothballed
retired
===regional calculations===
proposed
construction
shelved
cancelled
operating
idle
mothballed
retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_88092/2442660946.py:40: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_region.to_excel(excel_writer, 'Kilometers by region')
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_88092/2442660946.py:41: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_country.to_excel(excel_writer, 'Kilometers by country')


In [12]:
km_by_region

proposed  construction  \
Region Subregion                                    
Africa Northern Africa        68.50         100.0   
       Sub-Saharan Africa   6482.26        1950.0   
Total                       6550.76        2050.0   

                           in development (proposed + construction)  shelved  \
Region Subregion                                                               
Africa Northern Africa                                       168.50   488.00   
       Sub-Saharan Africa                                   8432.26   498.13   
Total                                                       8600.76   986.13   

                           cancelled  operating idle mothballed retired  
Region Subregion                                                         
Africa Northern Africa        620.04   16395.80                          
       Sub-Saharan Africa    2100.00    7457.94                          
Total                        2720.04   23853.74

## pipeline km by parent company (owner) and project status

### first check that there are no missing projectids

In [13]:
owner_parent_calculations_df = pandas.DataFrame()
# needs country, km in each country columns as well

for idx,row in country_ratios_df_subset.iterrows():
    #print(row.ProjectID)
    parent_string = row.Parent
    #print(parent_string)

    #print(parent_string)
    # if the first letter of the parent_string is a chinese character, and it ends with [100.00%],
    # that means it's a non-researched QCC owner; keep as-is
    if re.findall(r'[\u4e00-\u9fff]+', parent_string[:1]) != [] and parent_string[-9:]=='[100.00%]':
        parent_list = [parent_string.split(' [100.00%]')[0]]
        percent_list = [1.0]
    # otherwise do the rest of the thing
    else:
        parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
        percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        #print(parent_list)
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent,'ParentHQCountry'],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent,'ParentHQRegion'],
                                                                         'Country':row.Country,
                                                                         'Status':row.Status,
                                                                         'LengthMergedKmByCountry':row.LengthMergedKmByCountry}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.LengthMergedKmByCountry

In [14]:
owner_parent_calculations_df

,Parent,ProjectID,FractionOwnership,ParentHQCountry,ParentHQRegion,Country,Status,LengthMergedKmByCountry,KmOwnership
0,Sonatrach,P0524,1.00,Algeria,Africa,Algeria,operating,821.00,821.000
0,Sonatrach,P0525,1.00,Algeria,Africa,Algeria,operating,646.00,646.000
0,PipeChina,P0527,1.00,China,Asia,Chad,operating,300.00,300.000
0,Savannah Energy PLC,P0528,0.40,United Kingdom,Europe,Cameroon,operating,855.04,342.016
0,Petronas,P0528,0.35,Malaysia,Asia,Cameroon,operating,855.04,299.264
...,...,...,...,...,...,...,...,...,...
0,Libyan National Oil Corporation,P6443,1.00,Libya,Africa,Libya,operating,726.00,726.000
0,Libyan National Oil Corporation,P6444,0.50,Libya,Africa,Libya,operating,51.00,25.500
0,Libyan Investment Authority,P6444,0.25,Libya,Africa,Libya,operating,51.00,12.750
0,MedcoEnergi,P6444,0.25,Indonesia,Asia,Libya,operating,51.00,12.750


In [15]:
#unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = \
    owner_parent_calculations_df.groupby(
    ['Parent','ParentHQCountry','Status'])[['KmOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_km_by_status_df = owners_km_by_status_df.reindex(columns=status_list)
owners_km_by_status_df = owners_km_by_status_df.reset_index().set_index('Parent')
owners_km_by_status_df.columns.name = None

owners_km_by_status_df['in development (proposed + construction)'] = owners_km_by_status_df[['proposed','construction']].sum(axis=1)

owners_km_by_status_df = owners_km_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_km_by_status_df.loc['Total',:] = owners_km_by_status_df.sum(axis=0, min_count=0).values
owners_km_by_status_df.loc['Total','Country'] = ''

owners_km_by_status_df = owners_km_by_status_df.replace(numpy.nan, '')
owners_km_by_status_df = owners_km_by_status_df.replace(0, '')

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Kilometers by owner')

In [16]:
owners_km_by_status_df

,Country,proposed,construction,in development (proposed + construction),shelved,cancelled,operating,idle,mothballed,retired
Parent,,,,,,,,,,
APA Corporation,United States,,,,,,176.838,,,
Africa Oil,Canada,482.065,,482.065,,,,,,
Aiteo Group,Nigeria,,,,,,43.65,,,
CNOOC Limited,China,115.52,,115.52,,500.0,,,,
Carlyle Group,United States,,,,,,120.0,,,
Chevron,United States,,,,,,359.0,,,
China National Petroleum Corporation,China,,1950.0,1950.0,,300.0,640.0,,,
China Petrochemical Corporation,China,,,,,,46.662,,,
ConocoPhillips,United States,,,,,,155.931904,,,


### pipeline km by start year, type

In [17]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel'].isin(fuel_options))]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel'].isin(fuel_options))]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel'].isin(fuel_options))]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [18]:
pipes_proposed = pipes_df_touse.copy()
#pipes_proposed['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_proposed = pipes_proposed[(pipes_proposed['Status'].isin(['proposed'])) &
                              (pipes_proposed['Fuel'].isin(fuel_options))]
if fuel_type == 'Oil':
    pipes_proposed = pipes_proposed[(pipes_proposed['Status'].isin(['proposed'])) &
                              (pipes_proposed['Fuel'].isin(fuel_options))]
if fuel_type == 'NGL':
    pipes_proposed = pipes_proposed[(pipes_proposed['Status'].isin(['proposed'])) &
                              (pipes_proposed['Fuel'].isin(fuel_options))]

pipes_proposed_sum = pipes_proposed.groupby('ProposalYear')['LengthMergedKm'].sum()

In [19]:
pipes_construction = pipes_df_touse.copy()
#pipes_construction['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_construction = pipes_construction[(pipes_construction['Status'].isin(['construction'])) &
                              (pipes_construction['Fuel'].isin(fuel_options))]
if fuel_type == 'Oil':
    pipes_construction = pipes_construction[(pipes_construction['Status'].isin(['construction'])) &
                              (pipes_construction['Fuel'].isin(fuel_options))]
if fuel_type == 'NGL':
    pipes_construction = pipes_construction[(pipes_construction['Status'].isin(['construction'])) &
                              (pipes_construction['Fuel'].isin(fuel_options))]

pipes_construction_sum = pipes_construction.groupby('ConstructionYear')['LengthMergedKm'].sum()

In [20]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)))
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km operating'] = pipes_started_sum
    km_by_start_year['Oil pipeline km construction'] = pipes_construction_sum
    km_by_start_year['Oil pipeline km proposed'] = pipes_proposed_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

km_by_start_year.loc['Total',:] = km_by_start_year.sum(axis=0)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')
#km_by_start_year

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_88092/3403803826.py:23: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')


## save excel file

In [21]:
excel_writer.close()

## calculating stats for landing page

In [22]:
# number of projects tracked in total
print(pipes_df_touse.loc[pipes_df_touse.Fuel.str.contains(fuel_type)].shape[0], fuel_type, 'pipeline projects tracked')
print(pipes_df_touse.loc[pipes_df_touse.Fuel.str.contains(fuel_type)]['LengthMergedKm'].sum()/1e6, 'M km tracked')

116 Oil pipeline projects tracked
0.038334400000000005 M km tracked


In [23]:
# number of projects tracked in total
print(pipes_df_touse.loc[(pipes_df_touse.Fuel.str.contains(fuel_type))&
                        (pipes_df_touse.Status.isin(['proposed','construction']))].shape[0], fuel_type, 'pipeline projects tracked')
print(pipes_df_touse.loc[(pipes_df_touse.Fuel.str.contains(fuel_type))&
                        (pipes_df_touse.Status.isin(['proposed','construction']))]['LengthMergedKm'].sum()/1e3, 'K km tracked')

10 Oil pipeline projects tracked
9.30076 K km tracked


In [699]:
pipes_df_touse.Fuel.unique()

array(['Oil', 'Oil, NGL', 'Oil, NGL, naphtha'], dtype=object)